# Analysis of fitted tracks
Plots many distributions associated with fitted tracks.

## Paths
Set ``datapath`` to point to the directory which contains ``evd`` files.

In [1]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import h5py

In [2]:
datapath = os.path.join(os.environ['SCRATCH'],'evd/')
# datapath = '/global/project/projectdirs/dune/data/larpix/processed_data/prod_20_10_15/'

## Interactive analysis

In [10]:
%matplotlib widget
files = sorted([os.path.basename(path) for path in glob.glob(datapath+'/*.h5')])
@widgets.interact
def display(filename=widgets.Dropdown(options=files, description="File"),
            nhit_cut=widgets.IntSlider(value=0, min=0, max=100, step=1, description="NHit cut"),
            length_cut=widgets.FloatSlider(value=0, min=0, max=440, step=4.4, description="Length cut"),
            residual_cut=widgets.FloatSlider(value=300, min=0, max=300, step=3, description="Residual cut"),
            theta_cut=widgets.FloatSlider(value=0, min=0, max=np.pi/2, step=0.015, description="Theta cut")
           ):
    plt.close('all')
    f = h5py.File(os.path.join(datapath,filename),'r')
    if len(f['tracks']):            
        good_track_mask = np.ones(len(f['tracks'])).astype(bool)
        good_track_mask = np.logical_and(f['tracks']['nhit'] > nhit_cut, good_track_mask)
        good_track_mask = np.logical_and(f['tracks']['length'] > length_cut, good_track_mask)
        good_track_mask = np.logical_and(f['tracks']['residual'] < residual_cut, good_track_mask)
        good_track_mask = np.logical_and(np.abs(f['tracks']['theta']) > theta_cut, good_track_mask)
        good_track_mask = np.logical_and(np.abs(f['tracks']['theta'] - np.pi) > theta_cut, good_track_mask)
            
        plt.figure('track nhit')
        plt.hist(f['tracks']['nhit'],bins=np.linspace(0,100,100),histtype='step')
        plt.hist(f['tracks']['nhit'][good_track_mask],bins=np.linspace(0,100,100),histtype='step')
        plt.xlabel('nhit')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track length')
        plt.hist(f['tracks']['length'],bins=np.linspace(0,440,100),histtype='step')
        plt.hist(f['tracks']['length'][good_track_mask],bins=np.linspace(0,440,100),histtype='step')
        plt.xlabel('track length [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track theta')
        plt.hist(f['tracks']['theta'],bins=np.linspace(0,np.pi,100),histtype='step')
        plt.hist(f['tracks']['theta'][good_track_mask],bins=np.linspace(0,np.pi,100),histtype='step')
        plt.xlabel('track theta [rad.]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track phi')
        plt.hist(f['tracks']['phi'],bins=np.linspace(-np.pi,np.pi,100),histtype='step')
        plt.hist(f['tracks']['phi'][good_track_mask],bins=np.linspace(-np.pi,np.pi,100),histtype='step')
        plt.xlabel('track phi [rad.]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track res')
        plt.hist(f['tracks']['residual'],bins=np.linspace(0,100,100),histtype='step')
        plt.hist(f['tracks']['residual'][good_track_mask],bins=np.linspace(0,100,100),histtype='step')
        plt.xlabel('track residual [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track q')
        plt.hist(f['tracks']['q'],bins=np.linspace(0,5e3,100),histtype='step')
        plt.hist(f['tracks']['q'][good_track_mask],bins=np.linspace(0,5e3,100),histtype='step')
        plt.xlabel('track q [ke]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track dQ/dx')
        dqdx = f['tracks']['q']/f['tracks']['length']
        plt.hist(dqdx,bins=np.linspace(0,30,100),histtype='step')
        plt.hist(dqdx[good_track_mask],bins=np.linspace(0,30,100),histtype='step')
        plt.xlabel('track mean dQ/dx [ke/mm]')
        plt.legend(('all tracks','selected tracks'))
        
#         good_track_mask = np.ones(len(f['tracks'])).astype(bool)
        
        plt.figure('track dQ/dx v. theta')
        plt.hist2d(dqdx[good_track_mask],f['tracks']['theta'][good_track_mask],bins=(np.linspace(0,30,100),np.linspace(0,np.pi,100)))
        plt.colorbar()
        plt.xlabel('track mean dQ/dx [ke/mm]')
        plt.ylabel('track theta [rad.]')
        
        plt.figure('track dQ/dx v. phi')
        plt.hist2d(dqdx[good_track_mask],f['tracks']['phi'][good_track_mask],bins=(np.linspace(0,30,100),np.linspace(-np.pi,np.pi,100)))
        plt.colorbar()
        plt.xlabel('track mean dQ/dx [ke/mm]')
        plt.ylabel('track phi [rad.]')
        
        plt.figure('track dQ/dx v. residual')
        plt.hist2d(dqdx[good_track_mask],f['tracks']['residual'][good_track_mask],bins=(np.linspace(0,30,100),np.linspace(0,25,100)))
        plt.colorbar()
        plt.xlabel('track mean dQ/dx [ke/mm]')
        plt.ylabel('track residual [mm]')
        
        plt.figure('track residual v. theta')
        plt.hist2d(f['tracks']['residual'][good_track_mask],f['tracks']['theta'][good_track_mask],bins=(np.linspace(0,25,100),np.linspace(0,np.pi,100)))
        plt.colorbar()
        plt.xlabel('track residual [mm]')
        plt.ylabel('track theta [rad.]')
        
        plt.figure('track length v. theta')
        plt.hist2d(f['tracks']['length'][good_track_mask],f['tracks']['theta'][good_track_mask],bins=(np.linspace(0,440,100),np.linspace(0,np.pi,100)))
        plt.colorbar()
        plt.xlabel('track length [mm]')
        plt.ylabel('track theta [rad.]')
    else:
        print('no events in file')

interactive(children=(Dropdown(description='File', options=('datalog_2020_10_12_16_24_35_PDT_evd.h5', 'datalog…